---
title: 'Modeling Interactions Data with Boosted Trees'
subtitle: "Case Study Key"
author: "LASER Institute"
date: today 
format:
  html:
    toc: true
    toc-depth: 4
    toc-location: right
theme:
  light: simplex
  dark: cyborg
editor: visual
# bibliography: lit/references.bib
resource_files:
- img/tidymodels.png
jupyter: python3
---

```{r setup, include=FALSE}
knitr::opts_chunk$set(echo = TRUE, eval = TRUE)
```


## 1. PREPARE

After interpreting our last model, it is easy to think we can do a little better. But, how? In this module, we'll do do things: 1) specify a more sophisticated model and 2) go deeper on some feature engineering.

Our driving question for this module, then, is: **How much can we improve our model?** Looking back to our predictive model from module 3, we can see that our accuracy was okay. Can we improve on that? Let's dive in! First, we'll briefly review two key concepts for the module.

### A more sophisticated model

we attempt to improve our predictive performance by switching to a more complex model than the logistic regression we have been using–specifically, to a type of model that is based on decision trees – a boosted‑tree model. In short, this is one way we can improve our model's predictions.

Boosted trees (gradient boosting) build a *sequence* of small "decision trees" (detailed in the conceptual overview), each one focusing on the records the previous trees struggled with. When tuned carefully (small learning rate, many trees) the ensemble can outperform other algorithms based on decision trees you may have heard of, like the random forest algorithm.

### A little background (and some readings) on feature engineering

Feature engineering is a rich topic in machine learning research, including in the learning analytics and educational data mining communities.

Consider research on online learning and the work of Rodriguez et al. (2021). In this study, *digital trace data*, data generated through users' interactions with digital technologies. Optionally, review this paper -- specifically how they processed the "clickstream" data. As this paper illustrates, there is not one way to use such data.

> Rodriguez, F., Lee, H. R., Rutherford, T., Fischer, C., Potma, E., & Warschauer, M. (2021, April). Using clickstream data mining techniques to understand and support first-generation college students in an online chemistry course. In *LAK21: 11th International Learning Analytics and Knowledge Conference* (pp. 313-322).

Notably, the authors took several steps to prepare the data so that it could be validly interpreted. The same is true here in the context of machine learning. In a different context, the work of Gobert et al. (2013) is a great example of using data from educational simulations. Optionally review this paper, too, focused on their use of a technique they called *replay tagging* to conduct feature engineering.

> Gobert, J. D., Sao Pedro, M., Raziuddin, J., & Baker, R. S. (2013). From log files to assessment metrics: Measuring students' science inquiry skills using educational data mining. *Journal of the Learning Sciences, 22*(4), 521-563.

Even after feature engineering, machine learning approaches can often (but not always) be improved by choosing a more sophisticated model type. Note how we used a regression model in the first two case studies; here, we explore a considerably more sophisticated model, a boosted tree. Feature engineering and choosing a more sophisticated model adds some complexity to the modeling. As we have discussed, it is easy to bias our results if we repeatedly check the performance of *different model fits* with the same test data. Cross-validation is one commonly used solution for this problem.

## 2. WRANGLE

## Step 0: Loading and setting up

First, let's load the packages we'll use---the familiar pandas and several others focused on modeling.

#### **👉 Your Turn** **⤵**

Add to the chunk below code to load:

-   **pandas** for data wrangling (as "pd")
-   **matplotlib.pyplot** (as "plt")

Load these with the import function, per usual.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

Next we'll load a few more libraries for modeling and analysis.


In [ ]:
import statsmodels.formula.api as smf

from sklearn.model_selection import StratifiedKFold, cross_validate, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, cohen_kappa_score, make_scorer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

Let's get started! We have to do the same processing we did in the third module to obtain cut-off dates. The purpose of this is to train the model on data from the first one-third of the class, with the reasoning being this is a good time to intervene—-far enough into the class to make an appreciable impact, but not too late to have a limited chance of being able to change students' trajectory in the class. We'll repeat the procedure we carried out with the assessments data: calculating a cut-off for each class and then filtering the data based upon this cut-off.

First, though, we need to load *a really big file* with all of the interactions data — one with interactions (or log-trace) data. In the OULAD documentation, this is called the VLE (virtual learning environment) data source. Please review the description of the variables in the *studentVLE* and *VLE* sources (which are joined together for this module) [here](https://analyse.kmi.open.ac.uk/open_dataset#description).

The data is in a file named `oulad-interactions.parquet` file. You’ll find this in the `data` folder. Why this format? Parquet is a modern, columnar storage format that is highly efficient for large datasets. It allows for compressed storage and faster reading, especially when you only need a subset of columns. If we had stored this same data in a plain-text format like CSV, it would have taken up significantly more space — potentially too much to include in this lab's repository at all.

The tradeoff is that Parquet files aren’t immediately human-readable and can’t be opened directly in Excel. However, for programmatic analysis with tools like Python and Pandas, it’s an excellent option: compact, fast, and well-suited for large-scale data.

Now let’s load it:


In [ ]:
interactions_df = pd.read_parquet("data/oulad-interactions.parquet")

We can see that this file is indeed large - more than 10 million rows, with data on the number of *clicks* (`sum_clicks)` a student in a particular offering of a course makes with a particular component of the course (`id_site`) and the type of that component (`activity_type`). at a specific time.

Since you did something similar in the last module, you'll just run the code in the chunk below to calculate and filter the data based on a cut-off point (and associated date).


In [ ]:
assessments_df = pd.read_csv("data/oulad-assessments.csv")

code_module_dates_df = (
    assessments_df
    .groupby(['code_module', 'code_presentation'])['date']
    .quantile(0.25)
    .reset_index(name='quantile_cutoff_date')  # this just cleans up the naming of the summary statistic
)

interactions_joined_df = pd.merge(
    left=interactions_df, 
    right=code_module_dates_df,
    on=['code_module', 'code_presentation'],
    how='left'
)

interactions_filtered_df = interactions_joined_df.query('date < quantile_cutoff_date')

#### **👉 Your Turn** **⤵**

Now, use the `head()` method on the filtered data set you just created.


In [ ]:
interactions_filtered_df.head()

What do you notice about this data in relation to the original data set? You may wish to `info()` (or inspect in another way) the not filtered interactions data, too. Share a few notes on what you observe about these:

-   

-   

We will not only use this (filtered) interactions data, but also data on students and assessments. Here, we'll be expeditious by loading a file we created in the last module - `oulad-students_and_assessments.csv`. That is in this module's data folder; please load this in with `read_csv()`, as below.

#### **👉 Your Turn** **⤵**


In [ ]:
students_and_assessments_df = pd.read_csv("data/oulad-students-and-assessments.csv")

## 3. EXPLORE

#### **👉 Your Turn** **⤵**

*First*, use the `value_counts()` method on the `activity_type` variable in your filtered interactions data


In [ ]:
interactions_filtered_df['activity_type'].value_counts()

What does this tell you? Consulting the codebook and your output, please add at least two notes on what you are noticing:

-   The most common activity type is "forumng" with over 10 million interactions
-   Other common activities include subpage, oucontent, homepage, resource, and url interactions

#### **👉 Your Turn** **⤵**

*Second*, please create a histogram of the `date` variable.


In [ ]:
plt.hist(interactions_filtered_df['date'], edgecolor='white')
plt.xlabel('Date')
plt.ylabel('Count')
plt.title('Histogram of Interactions by Date')
plt.show()

What does this tell you? Add one or more notes:

-   The distribution shows interaction patterns over time, with some peaks and valleys suggesting different activity levels throughout the course

#### **👉 Your Turn** **⤵**

*Third*, please conduct one other data exploration step of your choosing. Options include creating simple graphs or calculating descriptive, summary statistics.


In [ ]:
interactions_filtered_df['sum_click'].describe().round()

We're ready to proceed to engineering some features with the interactions data. For the present time, we'll focus on the `sum_click` variable, which tells us how many times students clicked on a resource for a given date.

How can we create a feature with `sum_click`? Think back to our discussion in the presentation; we have *many* options for working with such time series data. Perhaps the most simple is to count the clicks. Please summarize the number of clicks for each student (specific to a single course). This means you will need to group your data by `id_student`, `code_module`, and `code_presentation`, and then create a summary variable. Assign the resulting output the name `interactions_summarized_df`. You may find the documentation for `groupby()` and `agg()` to be helpful. That is available [here](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.core.groupby.DataFrameGroupBy.agg.html). A useful hint is to use named variables in the `agg` function; e.g., `agg(sum_clicks=('sum_click', 'sum'), ...)` to avoid renaming columns in a later step.

#### **👉 Your Turn** **⤵**


In [ ]:
interactions_summarized_df = (
    interactions_filtered_df
    .groupby(['id_student', 'code_module', 'code_presentation'], as_index=False)
    .agg(sum_clicks=('sum_click', 'sum'))
)

#### **👉 Your Turn** **⤵**

How many times did students click? Let's create a histogram to see. Please use `plt.hist` to visualize the distribution of the `sum_clicks` variable you just created. Turn to the [documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html) if you need a pointer!


In [ ]:
plt.hist(interactions_summarized_df['sum_clicks'], edgecolor='white', bins=25)
plt.xlabel('Clicks')
plt.ylabel('Count')
plt.show()

This is a good start - we've created our first feature based upon the log data, `sum_clicks`! What are some other features we can add? An affordance of using the `agg()` function in Python is we can create multiple summary statistics at once. Let's also calculate the standard deviation of the number of clicks as well as the mean. Please copy the code you wrote above into the code chunk below and then add these two additional summary statistics.

#### **👉 Your Turn** **⤵**


In [ ]:
interactions_summarized_df = (
    interactions_filtered_df
    .groupby(['id_student', 'code_module', 'code_presentation'], as_index=False)
    .agg(sum_clicks=('sum_click', 'sum'),
        std_clicks=('sum_click', 'std'),
        mean_clicks=('sum_click', 'mean'))
        )

We'll take one last step here -- creating individual *slopes* of students' clicks over time. This code is a bit more involved, and so is annotated for you below; feel free to modify and re-use this.

First, we use a custom function that can deal with cases where the fitting of the slopes fails (e.g., for students with only a single data point, or some other unanticipated issue with the model estimation).


In [ ]:
def get_slope_and_intercept(data: pd.DataFrame) -> pd.Series:  # These extra bits of info (e.g., "-> pd.Series:") are type hints -- a useful, optional tool that tells the function's user what comes in and goes out. 
    try:
        # Fit a linear model with sum_click as the response and date as the predictor
        model = smf.ols('sum_click ~ date', data=data).fit()
        # Extract the model parameters for the Intercept and the slope of the date predictor
        return pd.Series({
            'intercept': model.params['Intercept'],
            'slope': model.params['date']
        })
    except Exception:
        # When fitting a slope fails, return an empty Pandas Series
        return pd.Series({'intercept': None, 'slope': None})

**Note: this next step may take up to a minute or so! Please feel free to relax!**


In [ ]:
model_params_df = (
    interactions_filtered_df
    .groupby(['id_student', 'code_module', 'code_presentation'])
    .apply(get_slope_and_intercept)
    .reset_index()
)
model_params_df = model_params_df.round(4)  # Reduce the precision of the slopes and intercepts

Let's use `pd.merge()` to left join the slopes and intercepts into the `interactions_summarized_df` feature dataset. Name the new DataFrame `interactions_summarized_and_slopes_df`.

#### **👉 Your Turn** **⤵**


In [ ]:
interactions_summarized_and_slopes_df = pd.merge(
    interactions_summarized_df,
    model_params_df,
    on=['id_student', 'code_module', 'code_presentation'],
    how='left'
)

Let's join together all of our features into a single file. Please use `pd.merge()` to left join `students_and_assessments_df` and `interactions_summarized_and_slopes_df`, assigning the resulting output the name `students_assessments_and_interactions_df`.

#### **👉 Your Turn** **⤵**


In [ ]:
students_assessments_and_interactions_df = pd.merge(
    students_and_assessments_df,
    interactions_summarized_and_slopes_df,
    on=['id_student', 'code_module', 'code_presentation'],
    how='left'
)

Lots of joining! Sometimes, the hardest part of complex analyses lies in the preparation (and joining) of the data.

Let's take a look at a few rows of your final dataset. Use the `describe()` function. If the numbers are in scientific notation, consider adding the `round` function to your call; e.g., `.round(2)`.

#### **👉 Your Turn** **⤵**


In [ ]:
students_assessments_and_interactions_df.describe().round(2)

We're now ready to model!

## 4. MODEL

### Step 1. Split data

We'll follow the same steps we followed in modules #2 and #3, here. One difference - we'll use `students_assessments_and_interactions_df` instead of the data frame we used in those modules. Please port over the code you used in those modules here, changing the name of the data frame to the one we are now using.

We discuss this first step minimally as we have now carried out a step very similar to this in LL1 and LL2; return to the case study for those (especially LL1) for more on data splitting.

#### **👉 Your Turn** **⤵**


In [ ]:
train_df, test_df = train_test_split(
    students_assessments_and_interactions_df, 
    test_size=0.25, 
    stratify=students_assessments_and_interactions_df['pass'], 
    random_state=20240712,
)

X_train = train_df.drop(columns=['pass'])  # Keeps all columns except our outcome column 'pass'
y_train = train_df['pass']  # Isolates just the outcome column

X_test = test_df.drop(columns=['pass'])
y_test = test_df['pass']

There is a key difference that is next. In this step. we'll further process `` `X_train` ``, creating different subsets of the data, or folds of the data, that we can use to fit our model multiple times. The first step is creating a `StratifiedKFold` object that we can use later in a process called cross-validation.


In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=22790)

How does this work? `X_train` is sampled as many time as we set the `n_splits` parameter. What does this mean in practice? It means that we split our data into 5 equally sized “folds.” Each fold contains a mix of examples that mirrors the overall balance of our outcome variable (because we're using **stratified** folds).

During cross-validation, our model will train on 4 of the folds and test on the 1 remaining fold**.** This process is repeated 5 times, each time rotating which fold is used as the test set.

### Step 2: Engineer features and build preprocessing pipeline

Here, we'll carry out several feature engineering steps as part of a scikit-learn `Pipeline`.

-   [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html): normalizes numeric data to have a standard deviation of one and a mean of zero
-   [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) and [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html): impute missing values using a simple strategy or the nearest neighbors method
-   [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html): convert nominal data (e.g. character or factors) into one or more numeric binary model terms for the levels of the original data.

We will use all four of these in making a `Pipeline` for both categorical data and numerical data.


In [ ]:
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy='most_frequent')),
    ("onehot", OneHotEncoder(drop="first", handle_unknown="ignore")),
])

numerical_transformer = Pipeline(steps=[
    ("imputer", KNNImputer(n_neighbors=5)),
    ("scaler", StandardScaler()),
])

#### **👉 Your Turn** **⤵**

Following the pattern from the Module-3 case study, combine the categorical and numerical columns into one preprocessor with the `ColumnTransformer`.

The categorical features we will use in the model are: `disability`, `gender`, and `code_module`

The numerical features are: `date_registration`, `mean_weighted_score`, `sum_clicks`, `std_clicks`, `mean_clicks`, `intercept` and `slope`


In [ ]:
categorical_features = [
    'disability',
    'gender',
    'code_module',
]

numerical_features = [
    'date_registration',
    'mean_weighted_score',
    'sum_clicks',
    'std_clicks',
    'mean_clicks',
    'intercept',
    'slope',
]

preprocessor = ColumnTransformer(transformers=[
    ("cat", categorical_transformer, categorical_features),  
    ("num", numerical_transformer, numerical_features)
])

### Step 3: Specify the model and complete the Pipeline

#### **👉 Your Turn** **⤵**

Then, please *add the preprocessor to the scikit-learn `GradientBoostingClassifier` in a final `Pipeline` named `model_pipeline`*. We will start with some good baseline parameters for the classifier: `learning_rate=0.05, n_estimators=1000, max_depth=4`.


In [ ]:
model_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", GradientBoostingClassifier(learning_rate=0.05, n_estimators=1000, max_depth=4))
])

**Why those defaults?** **n_estimators**: many small steps (1000) **learning_rate**: 0.05 (smaller → less variance needs more trees) **max_depth**: shallow (4) to avoid over‑fitting

### Step 4: Fit model

In this step, we use the `cross_validate` function (instead of `cross_val_score`, which was used in the code-along) because it allows us to evaluate multiple performance metrics simultaneously during cross-validation. While `cross_val_score` is simpler and useful when you're only interested in a single scoring metric (like accuracy), it becomes limiting when you need a broader view of model performance. The trade-off is that `cross_validate` produces a more complex output, which often requires extra steps to process and interpret.


In [ ]:
scoring_methods = {
    'accuracy': 'accuracy', 
    'precision': 'precision',
    'recall': 'recall', 
    'cohen_kappa_score': make_scorer(cohen_kappa_score),
}

results = cross_validate(model_pipeline, X_train, y_train, cv=skf, scoring=scoring_methods)
results_mean = pd.DataFrame(results).mean() # extra output processing

Note that you have fit as many models as the value for `n_splits` that you specified earlier. So, this may take some time. Take a walk, grab a snack, or make a cup of tea!

#### **👉 Your Turn** **⤵**

Ok, your model run is complete — let’s take a look at the results.

Start by viewing the full output from cross_validate, which is stored in the results dictionary. This shows the metric scores (and optional timing info) for each fold of cross-validation. Type `results` below.


In [ ]:
results

Then, check the `results_mean` object to see the average value of each metric across all folds. This provides a quick summary of overall model performance:


In [ ]:
results_mean

We noted earlier that we may introduce bias when we evaluate different models against the same training data. The benefit of using the multiple folds - and fitting multiple models - is we can make changes to our features or model. Thus, if we decide to add a new feature, we can run the above steps, without concern about biasing our interpretation of how accurate our model is.

That is, we can be unconcerned about inadvertently introducing bias until we reach the last step - fitting our last model. We can do that using the familiar `fit()` function.


In [ ]:
model_pipeline.fit(X_train, y_train)

### Step 5: Interpret accuracy

Finally, let's calculate the metrics for our final fit. *These* are the values that you should report as final. First, we will calculate predictions on the test dataset.


In [ ]:
y_preds = model_pipeline.predict(X_test)

Here we can use the `calculate_metrics` method from previous case studies.


In [ ]:
def calculate_metrics(y_actual, y_preds) -> dict: 
    """Calculates a suite of model metrics and returns the values as a dictionary"""
    accuracy = accuracy_score(y_actual, y_preds)
    precision = precision_score(y_actual, y_preds, pos_label=1)
    kappa = cohen_kappa_score(y_actual, y_preds)
    
    true_negative, false_positive, false_negative, true_positive = confusion_matrix(y_actual, y_preds).ravel()
    
    sensitivity = true_positive / (true_positive + false_negative)  # or the sklearn.metrics recall_score function
    specificity = true_negative / (true_negative + false_positive)
    
    negative_predictive_value = true_negative / (true_negative + false_negative)

    return {
        "accuracy": accuracy,
        "sensitivity": sensitivity,
        "specificity": specificity,
        "precision": precision,
        "negative_predictive_value": negative_predictive_value,
        "cohen_kappa": kappa
    }

Apply the `calculate_metrics` function to the predictions (`y_pred`) and the actual values in the test data (`y_test`):

#### **👉 Your Turn** **⤵**


In [ ]:
calculate_metrics(y_actual=y_test, y_preds=y_preds)

## 5. COMMUNICATE

Another benefit of a boosted tree model is we can interpret variable importance metrics. Do that here with the following code.


In [ ]:
model = model_pipeline.named_steps['model']
importances = model.feature_importances_

# Get categorical and numerical names from the fitted preprocessor
preprocessor = model_pipeline.named_steps['preprocessor']
categorical_names = preprocessor.named_transformers_['cat'] .named_steps['onehot'].get_feature_names_out(categorical_features)
numerical_names = numerical_features
all_feature_names = list(categorical_names) + numerical_names

# Build output Series
feature_importances = pd.Series(importances, index=all_feature_names)
print(feature_importances.sort_values(ascending=False))

Please add two or more notes on what you notice about which variables (features) are important, focused on what you would say to someone in your audience about what the important variables in your model were.

-   The mean weighted score remains the most important predictor, similar to the random forest model
-   The interaction features (sum_clicks, mean_clicks, etc.) show varying levels of importance, indicating that student engagement patterns do contribute to predicting success

### 🧶 Knit & Check ✅

Congratulations - you've completed this case study! Consider moving on to the badge activity next.